In [ ]:
from datetime import datetime

import akshare as ak  # 升级到最新版
import backtrader as bt  # 升级到最新版
import matplotlib.pyplot as plt  # 由于 Backtrader 的问题，此处要求 pip install matplotlib==3.2.2
import pandas as pd
import pyfolio as pf
import quantstats as qs
import platform
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import font_manager

def set_chinese_font():
    system = platform.system()  # 获取操作系统名称
    if system == "Windows":
        # Windows 使用 SimHei
        rcParams['font.sans-serif'] = ['SimHei']
    elif system == "Darwin":
        # macOS 使用 PingFang SC
        rcParams['font.sans-serif'] = ['Heiti TC']
    else:
        # 其他系统提示用户自行安装字体
        raise EnvironmentError("Unsupported OS. Please install a suitable Chinese font.")

    # 解决负号显示问题
    rcParams['axes.unicode_minus'] = False

# 设置字体
set_chinese_font()
# 利用 AKShare 获取股票的后复权数据，这里只获取前 6 列
stock_hfq_df = ak.stock_zh_a_hist(symbol="600256", adjust="hfq").iloc[:, :7]
# 删除 `股票代码` 列
del stock_hfq_df['股票代码']
# 处理字段命名，以符合 Backtrader 的要求
stock_hfq_df.columns = [
    "date",
    "open",
    "close",
    "high",
    "low",
    "volume",
]
# 把 date 作为日期索引，以符合 Backtrader 的要求
stock_hfq_df.index = pd.to_datetime(stock_hfq_df["date"])

class MyStrategy(bt.Strategy):
    """
    主策略程序
    """
    # 设定交易策略的参数
    params = dict(
        maperiod=20,
    )
    def __init__(self):
        """
        初始化函数
        """
        # 初始化交易指令、买卖价格和手续费
        self.order = None
        self.buy_price = None
        self.buy_comm = None
        # 添加移动均线指标
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0].lines.close, period=self.params.maperiod
        )
        self.sma.csv = True

    def next(self):
        """
        执行逻辑
        """
        if self.order:  # 检查是否有指令等待执行，如果有未完成的订单则跳过
            return
        # 检查是否持仓
        if not self.position:  # 没有持仓
            # 执行买入条件判断：收盘价格上涨突破 20 日均线
            if self.datas[0].lines.close[-1] < self.sma[-1] and self.datas[0].lines.close[0] > self.sma[0]:
                self.order = self.buy(size=100)  # 执行买入
        # 执行卖出条件判断：收盘价格跌破 20 日均线
        elif self.datas[0].lines.close[-1] > self.sma[-1] and self.datas[0].lines.close[0] < self.sma[0]:
            self.order = self.sell(size=100)  # 执行卖出


In [ ]:
cerebro = bt.Cerebro()  # 初始化回测系统
start_date = datetime(2000, 4, 3)  # 回测开始时间
end_date = datetime(2021, 9, 12)  # 回测结束时间
data = bt.feeds.PandasData(
    dataname=stock_hfq_df, fromdate=start_date, todate=end_date
)  # 加载数据
cerebro.adddata(data)  # 将数据传入回测系统
cerebro.addstrategy(MyStrategy)  # 将交易策略加载到回测系统中

# 添加观察者 PyFolio
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='PyFolio')
# 添加书写者
cerebro.addwriter(bt.WriterFile, csv=True, out="result.csv")

start_cash = 1000000
cerebro.broker.setcash(start_cash)  # 设置初始资本为 100000
cerebro.broker.setcommission(commission=0.002)  # 设置交易手续费为 0.2%
result = cerebro.run()[0]  # 运行回测系统

port_value = cerebro.broker.getvalue()  # 获取回测结束后的总资金
pnl = port_value - start_cash  # 盈亏统计

print(
    f"初始资金: {start_cash}\n回测期间：{start_date.strftime('%Y%m%d')}:{end_date.strftime('%Y%m%d')}"
)
print(f"总资金: {round(port_value, 2)}")
print(f"净收益: {round(pnl, 2)}")

# cerebro.plot(style="candlestick")  # 画图

portfolio_stats = result.analyzers.getbyname('PyFolio')
returns, positions, transactions, gross_lev = portfolio_stats.get_pf_items()
print(returns)
pf.create_full_tear_sheet(returns)
qs.reports.full(returns)

qs.stats.sharpe(returns)
qs.stats.conditional_value_at_risk(returns)

help(qs.stats.conditional_value_at_risk)
qs.extend_pandas()
stock = qs.utils.download_returns('000001.SS')
stock.sharpe()
qs.plots.snapshot(stock['000001.SS'], title='Facebook Performance')
returns.index = returns.index.tz_convert(None)
qs.reports.html(returns, benchmark=stock['000001.SS'],benchmark_title="Facebook", output='stats.html', title='Report')  # 会在本地目录生成 html 文件，用游览器打开即可


In [ ]:
import matplotlib
from matplotlib import font_manager
 
font_list=sorted([f.name for f in matplotlib.font_manager.fontManager.ttflist])
for i in font_list:
    print(i)

In [ ]:
type(stock['000001.SS'])